In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# 파일 Read
def read_text(filename):
    results = []
    with open(f"../datasets/kbo_corpus/kia_{filename}.txt", 'r') as fr:
        samsung = fr.read()
    results.append(samsung.split('\n'))
    with open(f"../datasets/kbo_corpus/samsung_{filename}.txt", 'r') as fr:
        kia = fr.read()
    results.append(kia.split('\n'))
    with open(f"../datasets/kbo_corpus/lotte_{filename}.txt", 'r') as fr:
        lotte = fr.read()
    results.append(lotte.split('\n'))
    return results

In [6]:
cleaned_documents_morph = read_text('cleaned')
cleaned_documents_morph[0][:10]

['정해영/NNP 올림픽/NNG 가다/VV 한심/NNG 사토/NNG 털릴준비하다/VV',
 '작년/NNG 황대/NNG 기회/NNG 안준거/NNP 진짜/MAG 아깝/VA',
 '작년/NNG 머인/NNG 군무대/NNG 처음/NNG 타석이상/NNG 한/NNG 것/NNB',
 '이민우/NNP 데뷔승/NNG 이후/NNG 승/NNG',
 '야구장/NNG 가다/VV 라다/VV 케텍스/NNP 한참/NNG 가다/VV 야되다/VV 않다/VV',
 '소형준/NNG 신인/NNG 시즌/NNG 경기/NNG 방어/NNG 점/NNB 대/NNG',
 '올해/NNG 쓰다/VV 않다/VV 거/NNB 씹민상/NNG 왜/MAG 썻/NNG',
 '근데/MAJ 설다/VV 프로필/NNG 존나/NNG 사/NNG 잘/MAG 찍다/VV 주다/VV',
 '황대/NNG 페/NNG 이/NNP 스북/NNG',
 '그러다/VV 황대인/NNG 점점/MAG 공/NNG 맞히다/VV']

In [4]:
'''
import re
p = re.compile('[가-힣A-Za-z0-9]+/NN.')
mached_terms = re.findall(p, '정해영/NNP 올림픽/NNG 가다/VV 한심/NNG 사토/NNG 털릴준비하다/VV')
mached_terms
'''

['정해영/NNP', '올림픽/NNG', '한심/NNG', '사토/NNG']

In [7]:
# morph (/ 뒤의 성분) 제거, 명사 성분만 남김
import re
p = re.compile('[가-힣A-Za-z0-9]+/NN.')
cleaned_documents = []
for i in range(3):
    for sen in cleaned_documents_morph[i]:
        sentence = ''
        mached_terms = re.findall(p, sen) 
        for word in mached_terms:
            lemma = word.split('/')[0]
            sentence += lemma + ' '
        cleaned_documents.append(sentence.strip())

In [8]:
cleaned_documents[:10]

['정해영 올림픽 한심 사토',
 '작년 황대 기회 안준거',
 '작년 머인 군무대 처음 타석이상 한 것',
 '이민우 데뷔승 이후 승',
 '야구장 케텍스 한참',
 '소형준 신인 시즌 경기 방어 점 대',
 '올해 거 씹민상 썻',
 '프로필 존나 사',
 '황대 페 이 스북',
 '황대인 공']

In [38]:
# tf-idf 행렬 생성
vectorizer = TfidfVectorizer(max_features=5000, max_df = 0.5, smooth_idf=True)
X = vectorizer.fit_transform(cleaned_documents)
X.shape

(56561, 5000)

In [43]:
# 토픽 모델링 - Truncated SVD 이용, 차원 축소
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=500, random_state=42)
svd_model.fit(X)
len(svd_model.components_)    # VT의 행 개수 = topic 개수

10

In [44]:
import numpy as np
np.shape(svd_model.components_)

(10, 5000)

In [47]:
terms = vectorizer.get_feature_names()    # 단어 집합

# 각 topic의 1000개 단어 중 값이 큰 상위 n개를 출력
def get_topics(components, feature_names, n=5):
    # i : topic의 index, j : term의 index
    for i, topic in enumerate(components):
        print(f"Topic {i+1}:", [(feature_names[j], topic[j].round(5)) for j in topic.argsort()[:-(n+1):-1]])
        # 마지막 [:-(n+1):-1] -> sort된 numpy 결과값을 뒤집어 내림차순 정렬

In [48]:
get_topics(svd_model.components_, terms)
# topic modeling이 제대로 안 됨. 역시 데이터가 안 좋은가 보다.
# 그 와중에 욕 비중 무엇....... 익명 커뮤니티라서 그런가...?

Topic 1: [('오늘', 0.98334), ('경기', 0.14692), ('새끼', 0.06009), ('진짜', 0.02428), ('이유', 0.02334)]
Topic 2: [('새끼', 0.89661), ('진짜', 0.28383), ('시발', 0.26993), ('개추', 0.12763), ('요즘', 0.05484)]
Topic 3: [('시발', 0.84309), ('진짜', 0.36193), ('내일', 0.08098), ('경기', 0.04741), ('선발', 0.03099)]
Topic 4: [('진짜', 0.81852), ('내일', 0.25548), ('경기', 0.08824), ('선발', 0.07179), ('투수', 0.06144)]
Topic 5: [('내일', 0.89595), ('경기', 0.22458), ('선발', 0.18835), ('투수', 0.05542), ('야구', 0.0326)]
Topic 6: [('경기', 0.94598), ('시즌', 0.06334), ('작년', 0.06192), ('올해', 0.05015), ('어제', 0.03439)]
Topic 7: [('투수', 0.81163), ('야구', 0.38733), ('선발', 0.21934), ('선수', 0.20601), ('올해', 0.1621)]
Topic 8: [('야구', 0.77232), ('선수', 0.39408), ('요즘', 0.07512), ('올해', 0.05247), ('사람', 0.03195)]
Topic 9: [('올해', 0.94176), ('시즌', 0.11572), ('작년', 0.1109), ('이유', 0.07996), ('홈런', 0.06268)]
Topic 10: [('선발', 0.89285), ('시즌', 0.11726), ('선수', 0.0935), ('다음', 0.05337), ('의리', 0.04311)]
